# Twitter Sentiment Analysis 

# ARTG Project - By Team Maple

**<font color="blue">Import libraries, read the file and remove duplicates</font>**  

**<font color="red">Code:</font>**

In [7]:
import csv
import tweepy

ModuleNotFoundError: No module named 'tweepy'

In [ ]:
consumer_key = 'kmflAhDlFLYvVLzUhUTv5Agbw'
consumer_secret = 'D8Giq3GMjIMaBQN0Qw63tsoHpajA7hNKy2ouo1XeqQP46SP38C'
access_token = '4266439228-U0ySetwuTNEtz3ZGdyPWKVrfIGKY866EMIrzNbN'
access_token_secret = 'CVynCeCX22NtK8iygTHGWjota2JhGTfPgk2CO1MUnPfBK'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, timeout=1000)

In [ ]:
# Open/create a file to append data to
csvFile = open('resultgf2.csv', 'a')

# Use csv writer
csvWriter = csv.writer(csvFile)

for tweet in tweepy.Cursor(api.search,
                           q="#GeorgeFlyod",
                           since="2020-06-22",
                           until="2020-06-28",
                           lang="en",
                           count=100).items():
    # Write a row to the CSV file. I use encode UTF-8
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
    print(tweet.created_at, tweet.text)
csvFile.close()